## Modules à importer

In [1]:
import math
import scipy
import pickle
import numpy as np
import pandas as pd
import iminuit as im
import ipywidgets as ipw
from snprop import age
from snprop import survey
from variaIa import tools
from variaIa.tools import surveys, df_cons, df_nc, df_c, df_full
from variaIa import massevol
from importlib import reload
from variaIa import stretchevol
from matplotlib.lines import Line2D
from pprint import PrettyPrinter
import matplotlib.pyplot as plt
import random

## Définitions pandas, surveys et sélection de l'échantillon à utiliser

In [2]:
def set_survey(cons, su):
    global df
    if su == 'All':
        df = df_cons(cons)
    elif 'n' in su:
        df = df_cons(cons)[df_cons(cons)['survey'] != su.split('n')[-1]]
    else:
        df = df_cons(cons)[df_cons(cons)['survey'] == su]
    return df

int_set_su = ipw.interactive(set_survey, cons=tools.cons, su=tools.su)
display(int_set_su)

smap = plt.cm.get_cmap('cividis')
#smap = plt.cm.get_cmap('viridis')
colors = {'SNF': plt.cm.binary(0.9),
          'low-z': (1, 0, 0),
          'SDSS': smap(0.1),
          'PS1': smap(0.5),
          'SNLS': smap(0.8),
          'HST': smap(0.99),  # for cividis
          #'HST': smap(300),  # for viridis
          'All': (255/255, 51/255, 0/255)} 

def alphacolors(alpha, survey):
    return(colors[survey][0], colors[survey][1], colors[survey][2], alpha)

df_nHST = df_nc[df_nc['survey'] != 'HST']
df_nlowz = df_nc[df_nc['survey'] != 'low-z']

interactive(children=(Checkbox(value=False, description='Conservative'), Dropdown(description='Survey:', optio…

### Use SED fitted SNf data

In [3]:
sed = pd.read_csv('../../../Data/mass/SNFactory_SEDfit_Masses.txt')
sed = sed.loc[sed['name'].str.contains( 'SNF|LSQ|PTF', na=False, regex=True)]
sed = sed.rename(columns={'mass': 'hostmass', 'z': 'redshifts'})
sed['hostmass_err'] = np.sqrt((sed['mass_uppererr'].values**2 +
                               sed['mass_lowerr'].values**2)/2)
snf_df_sed = df_nc[df_nc['survey'] == 'SNF']
snf_df_sed['hostmass'] = sed['hostmass'].values
snf_df_sed['hostmass_err'] = sed['hostmass_err'].values
snf_sed_nonan = snf_df_sed[~np.isnan(snf_df_sed['hostmass'])]

/home/nicolas/.installs/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/home/nicolas/.installs/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


## Minimisation des modèles à mettre dans le _pandas_

In [8]:
gen = massevol.generic()
gen.set_model('Evol3G3M3S')
evol3G3M3S = gen.fit(df, limit_a=(0,1))

gen.set_model('Evol3G3M4S')
evol3G3M4S = gen.fit(df, limit_a=(0,1), limit_mu_2=(8, 10), limit_mu=(10, 11))

gen.set_model('Evol2G2M2S')
evol2G2M2S = gen.fit(df)

gen.set_model('Evol2G2M3S')
evol2G2M3S = gen.fit(df)

In [4]:
gen = massevol.generic()
gen.set_model('Evol4G4M4S')
evol4G4M4S = gen.fit(snf_sed_nonan,
                     limit_a=(0,1),
                     limit_b=(0,1))

gen.set_model('Evol3G3M4S')
evol3G3M4S = gen.fit(snf_sed_nonan, limit_a=(0, 1), limit_mu_2=(8, 10), limit_mu=(10, 11))

gen.set_model('Evol3G3M3S')
evol3G3M3S = gen.fit(snf_sed_nonan,
                     limit_a=(0,1),
                     limit_sigma_2=(0, 3))#,
#                     limit_sigma_3=(0.2, 3))

gen.set_model('Evol2G2M3S')
evol2G2M3S = gen.fit(snf_sed_nonan, limit_sigma_1=(0, 3))

gen.set_model('Evol2G2M2S')
evol2G2M2S = gen.fit(snf_sed_nonan)

gen.set_model('Evol1G1M2S')
evol1G1M2S = gen.fit(snf_sed_nonan)

gen.set_model('Evol1G1M1S')
evol1G1M1S = gen.fit(snf_sed_nonan)

mod_list = [evol1G1M1S, evol1G1M2S, evol2G2M2S, evol2G2M3S, evol3G3M3S, evol3G3M4S, evol4G4M4S]
mod_name = ['Evol'+mod.__doc__ for mod in mod_list]

pp = PrettyPrinter()
for mod in mod_list:
    print(mod.__doc__)
    pp.pprint(mod.param)

1G1M1S
{'mu': 9.821876257041122, 'sigma': 0.7456061857907911}
1G1M2S
{'mu': 10.193756829070729,
 'sigmadown': 0.9740459104736444,
 'sigmaup': 0.48906638377533973}
2G2M2S
{'mu_1': 9.322332747078347,
 'mu_2': 10.341604799509966,
 'sigma_1': 0.5819364913189441,
 'sigma_2': 0.5095830489262351}
2G2M3S
{'mu': 10.556046684125041,
 'mu_1': 9.336170329542629,
 'sigma_1': 0.5838939046267564,
 'sigmadown': 0.7030202152746594,
 'sigmaup': 0.34234808640384845}
3G3M3S
{'a': 0.9152024426792874,
 'mu_1': 9.332949614054824,
 'mu_2': 10.440669576076836,
 'mu_3': 9.105427163510777,
 'sigma_1': 0.5845593342819263,
 'sigma_2': 0.3802795789329329,
 'sigma_3': 6.3398188614309276e-06}
3G3M4S
{'a': 0.2939410586587106,
 'mu': 10.47561029174132,
 'mu_1': 9.337237869652864,
 'mu_2': 9.999997296046306,
 'sigma_1': 0.5767513397863515,
 'sigma_2': 0.7683027268314977,
 'sigmadown': 0.34469482671096024,
 'sigmaup': 0.33503679941988956}
4G4M4S
{'a': 0.33747456521674357,
 'b': 0.9123684990757337,
 'mu_1': 8.675593904966

In [8]:
evol3G3M3S.migrad_out

------------------------------------------------------------------
| FCN = 214.9                   |     Ncalls=559 (559 total)     |
| EDM = 1.67e-07 (Goal: 0.0002) |            up = 1.0            |
------------------------------------------------------------------
|  Valid Min.   | Valid Param.  | Above EDM | Reached call limit |
------------------------------------------------------------------
|     True      |     True      |   False   |       False        |
------------------------------------------------------------------
| Hesse failed  |   Has cov.    | Accurate  | Pos. def. | Forced |
------------------------------------------------------------------
|     False     |     True      |   True    |   True    | False  |
------------------------------------------------------------------
---------------------------------------------------------------------------------------------
|   | Name    |   Value   | Hesse Err | Minos Err- | Minos Err+ | Limit-  | Limit+  | Fixed |
---------------------------------------------------------------------------------------------
| 0 | mu_1    |   9.33    |   0.10    |            |            |         |         |       |
| 1 | sigma_1 |   0.58    |   0.07    |            |            |         |         |       |
| 2 | a       |   0.92    |   0.04    |            |            |    0    |    1    |       |
| 3 | mu_2    |   10.44   |   0.07    |            |            |         |         |       |
| 4 | sigma_2 |   0.38    |   0.06    |            |            |    0    |    3    |       |
| 5 | mu_3    |   9.11    |   0.05    |            |            |         |         |       |
| 6 | sigma_3 |   0e-2    |   6e-2    |            |            |         |         |       |
---------------------------------------------------------------------------------------------

## Get proba

In [5]:
def get_proba(model):
    return np.exp((evol3G3M3S.get_aic() - model.get_aic())/2)

## Comparison dataframe

In [7]:
modèles = [evol3G3M3S, evol2G2M3S, evol2G2M2S, evol4G4M4S, evol3G3M4S, evol1G1M2S, evol1G1M1S]

d_mod_comp = pd.DataFrame({'Name': [k.__doc__ for k in modèles],
                           'Fixed': [k.FIXED for k in modèles],
                           'Free param': [len(k.FREEPARAMETERS) for k in modèles],
                           '$\mathcal{L}$': [round(k.get_logl(),1) for k in modèles],
                           'AIC': [round(k.get_aic(),1) for k in modèles],
                           '$\Delta$ AIC': [round(evol3G3M3S.get_aic()-k.get_aic(),1) for k in modèles],
                           'Proba': [get_proba(k) for k in modèles]
                          })

path = '../../../Data/tables/mass_comp/mass_df-sed_comp'
if tools.cons.value:
    path += '_cons'
path += '.dat'
d_mod_comp.to_csv(path)

d_mod_comp

,Name,Fixed,Free param,$\mathcal{L}$,AIC,$\Delta$ AIC,Proba
0,3G3M3S,False,7,214.9,228.9,0.0,1.000000e+00
1,2G2M3S,False,5,221.4,231.4,-2.4,2.989049e-01
2,2G2M2S,False,4,223.9,231.9,-3.0,2.265120e-01
3,4G4M4S,False,10,213.0,233.0,-4.1,1.289009e-01
4,3G3M4S,False,8,219.6,235.6,-6.7,3.562666e-02
5,1G1M2S,True,3,259.8,265.8,-36.8,1.000123e-08
6,1G1M1S,True,2,262.4,266.4,-37.5,7.274462e-09
